In [1]:
import os
import pandas as pd
import numpy as np
import openpyxl

In [2]:
'''Guia da numeração das colunas'''
# Questionario socio economico:
# Colunas 0 a 11

# teste de compulsão alimentar: N  a AC:
# Colunas 12 a 26

# Ortorexia:
# Colunas 27 a 41

'Guia da numeração das colunas'

In [3]:
def clean_weight(value):
    if isinstance(value, str):  # Se for string
        value = value.replace('kg', '').replace('Kg', '').replace('KG', '').strip()  # Remove 'kg'
        value = value.replace(',', '.')  # Troca vírgula por ponto
        try:
            return float(value)  # Tenta converter para float
        except ValueError:
            return None  # Retorna None se a conversão falhar
    elif isinstance(value, (int, float)):  # Se já for numérico
        return float(value)  # Retorna como float
    else:
        return None  # Para outros tipos (e.g., None)

def clean_height(value):
    if isinstance(value, str):  # Se for string
        value = value.replace('cm', '').replace('m', '').replace(',', '.').strip()  # Remove 'cm' e 'm', troca vírgula por ponto
        try:
            return float(value)  # Tenta converter para float
        except ValueError:
            return None  # Retorna None se a conversão falhar
    elif isinstance(value, (int, float)):  # Se já for numérico
        if isinstance(value, int) and value > 100:  # Se for inteiro e maior que 100
            return value / 100.0  # Divide por 100
        return float(value)  # Retorna como float
    else:
        return None  # Para outros tipos (e.g., None)

In [4]:
file_path = "../data/filtered_data.csv"

if os.path.exists(file_path):
    filtered_df = pd.read_csv("../data/filtered_data.csv")
else:
    df = pd.read_excel("../data/raw_data .xlsx", engine='openpyxl')

    # Filtra se o consentimento foi dado e remvoe a coluna de consentimento (só por garantia)
    consentiment_column = df.columns[1]
    filtered_df = df[df[consentiment_column] == 'Li e concordo em participar da pesquisa.'].drop(['Carimbo de data/hora', 'Cidade e Estado:', consentiment_column], axis =1)
    
    # Renomeando algumas colunas:
    filtered_df.columns.values[0] = 'idade'
    filtered_df.columns.values[1] = 'peso'
    filtered_df.columns.values[2] = 'altura'
    filtered_df.columns.values[3] = 'sexo'
    filtered_df.columns.values[4] = 'gênero'
    filtered_df.columns.values[5] = 'etnia'
    q = 1
    for i in range(len(filtered_df.columns)):
        if i >= 26:

            filtered_df.columns.values[i] = str(q)
            q += 1

    # Tratando coluna da idade:    
    filtered_df['idade'] = filtered_df['idade'].apply(lambda x: ''.join(filter(str.isdigit, str(x))) if pd.notna(x) else x)
    filtered_df['idade'] = pd.to_numeric(filtered_df['idade'], errors='coerce').astype('Int64')

    # Tratando coluna peso:
    # Aplicar a função de limpeza
    filtered_df['peso'] = filtered_df['peso'].apply(clean_weight)

    # Tratando coluna altura:
    filtered_df['altura'] = filtered_df['altura'].apply(clean_height)

    filtered_df.to_csv("../data/filtered_data.csv", index=False)

In [5]:
def check_compulsion(row):
    similar_questions = ['#2','#5', '#8', '#9','#10','#11','#12','#14','#15',]
    similar_answers = {1:0, 2:1, 3:2, 4:3}
    other_questions = {
        '#1': {1:0, 2:0, 3:1, 4:3},
        '#3': {1:0, 2:1, 3:3, 4:3},
        '#4': {1:0, 2:0, 3:0, 4:2},
        '#6': {1:0, 2:1, 3:3, 4:None},
        '#7': {1:0, 2:2, 3:3, 4:3},
        '#13': {1:0, 2:0, 3:2, 4:3},
        '#16': {1:0, 2:1, 3:2, 4:None},
    }
    total = 0
    answers_profile = []
    for column in row.columns:
        if column in similar_questions:
            user_answer = int(row[column].values[0])
            value = similar_answers[user_answer]
        else:
            user_answer = int(row[column].values[0])
            value = other_questions[column][user_answer]
        answers_profile.append([column, user_answer, value])
        total += value
    return total, answers_profile

def check_ortorexia(row):
    similar_questions_01 = ['2','5','8','9']
    similar_answers_01 = {'Sempre': 4, 'Muitas vezes': 3, 'Algumas vezes': 2, 'Nunca': 1}

    similar_questions_02 = ['3','4','6','7','10','11','12','14','15',]
    similar_answers_02 = {'Sempre': 1, 'Muitas vezes': 2, 'Algumas vezes': 3, 'Nunca': 4}

    similar_questions_03 = ['1','13']
    similar_answers_03 = {'Sempre': 2, 'Muitas vezes': 4, 'Algumas vezes': 3, 'Nunca': 1}

    total = 0
    for column in row.columns:
        if column in similar_questions_01:
            user_answer = row[column].values[0]
            value = similar_answers_01[user_answer] 
        elif column in similar_questions_02:
            user_answer = row[column].values[0]
            value = similar_answers_02[user_answer] 
        elif column in similar_questions_03:
            user_answer = row[column].values[0]
            value = similar_answers_03[user_answer]
        total += value
    return total

def check_nutrition_status(peso, altura):
    nutrition_status_cases = {
        'Baixo peso': 1,
        'Peso normal': 2,
        'Sobrepeso': 3,
        'Obesidade grau I': 4,
        'Obesidade grau II': 5,
        'Obesidade grau III': 6,
    }

    imc = peso/(altura)**2
    nutrition_status = ""
    if imc < 18.5:
       nutrition_status =  'Baixo peso'
    elif imc >= 18.5 and imc < 25:
       nutrition_status =  'Peso normal' 
    elif imc >= 25 and imc < 30:
        nutrition_status =  'Sobrepeso'
    elif imc >= 30 and imc < 35:
        nutrition_status =  'Obesidade grau I'
    elif imc >= 35 and imc < 40:
        nutrition_status = 'Obesidade grau II'
    elif imc >= 40:
        nutrition_status =  'Obesidade grau III'
    return nutrition_status, nutrition_status_cases[nutrition_status]

In [6]:
# Iterando o df linha alinha (usuario a usuario)
df = filtered_df.copy(deep=True)
size = df.shape[0]

new_df = pd.DataFrame()
for i in range(size):
    row =  filtered_df.iloc[i].to_frame().transpose()

    # Dados socio-econômicos
    row_social = row.iloc[:, 0:10]       # 0:11 includes columns from position 0 to 10
    
    idade = row_social['idade'].values[0]
    peso = row_social['peso'].values[0]
    altura = row_social['altura'].values[0]
    
    # Dados de Compulsão alimentar
    row_compulsion = row.iloc[:, 10:26]   # 10:26 includes columns from position 10 to 25
    row_compulsion = row_compulsion.apply(lambda col: col.str.replace(r'[^1234]', '', regex=True))
    total_compulsion, _ = check_compulsion(row_compulsion)
    row['indice_compulsao'] = [total_compulsion]
    
    # Dados Ortorexia
    row_ortorexia = row.iloc[:, 26:41]   # 26:41 includes columns from position 26 to 40
    row_ortorexia.columns = row_ortorexia.columns.str.extract(r'(\d+)')[0]
    total_ortorexia = check_ortorexia(row_ortorexia)
    row['indice_ortorexia'] = [total_ortorexia]

    # Dados estado nutricional:
    nutrition_status, nutrition_status_ref  = check_nutrition_status(peso, altura)
    row['estado_nutricional'] = [nutrition_status]
    row['estado_nutricional_ref'] = [nutrition_status_ref]
    new_df = pd.concat([new_df, row])

new_df.to_excel('../data/final_data_output.xlsx', index=False, engine='openpyxl')
